In [ ]:
# Importaciones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import joblib

### Preprocesamiento de los datos

In [ ]:
# Carga de datasets con datos de partida limpia
df_2024 = pd.read_csv('datos__2024.csv', parse_dates=['Timestamp'], sep=';')
df_2025 = pd.read_csv('datos__2025.csv', parse_dates=['Timestamp'], sep=';')
df_2025_last = pd.read_csv('datos__2025_last.csv', parse_dates=['Timestamp'])
df_actual = pd.read_csv(r"C:\Users\nicob\Downloads\datos_nuevos.csv", parse_dates=['Timestamp'])

# Convertir columna 'Timestamp' a tipo datetime para eliminar la información de zona horaria (tz)
df_2024['Timestamp'] = pd.to_datetime(df_2024['Timestamp'])
df_2024['Timestamp'] = df_2024['Timestamp'].dt.tz_localize(None)
df_2025['Timestamp'] = pd.to_datetime(df_2025['Timestamp'])
df_2025['Timestamp'] = df_2025['Timestamp'].dt.tz_localize(None)
df_2025_last['Timestamp'] = pd.to_datetime(df_2025_last['Timestamp'])
df_2025_last['Timestamp'] = df_2025_last['Timestamp'].dt.tz_localize(None)
df_actual['Timestamp'] = pd.to_datetime(df_actual['Timestamp'])
df_actual['Timestamp'] = df_actual['Timestamp'].dt.tz_localize(None)

# Unir y eliminar registros duplicados entre los datasets
df = pd.concat([df_2024, df_2025, df_2025_last, df_actual])
df = df.drop_duplicates(subset=['Timestamp'], keep='last')
df.sort_values('Timestamp', inplace=True)

In [ ]:
# Filtrar datos para generar archivo adicional de testing final
fecha_test_final = pd.to_datetime('2025-04-01')
df_test_final = df[df['Timestamp'] > fecha_test_final].reset_index(drop=True)

In [ ]:
# Filtrar datos de entrenamiento de diciembre en adelante (Partida Limpia)
fecha_inicio = pd.to_datetime('2024-12-20')
df = df[(df['Timestamp'] >= fecha_inicio) & (df['Timestamp'] <= fecha_test_final)].reset_index(drop=True)

In [ ]:
df.to_csv('datos_diciembre_marzo_entrenamiento.csv', index=False)

In [ ]:
# Creacion de columnas auxiliares para filtrado de datos operacionales
df['RUN_SHIFT'] = df['CNN-3200-CR_0001_MO.RUN'].shift(1, fill_value=0)  # Estado anterior de RUN del motor
df['CAMBIO_RUN'] = (df['RUN_SHIFT'] == 0) & (df['CNN-3200-CR_0001_MO.RUN'] == 1)  # Detectar cambios de 0 a 1 en el RUN del motor

# Crear una columna de omisión para los primeros 8 registros después del cambio
df['OMITIR'] = df['CAMBIO_RUN'].rolling(window=8, min_periods=1).max()

# Filtrar dataset con los datos operativos (motor en funcionamiento, presencia de material de chancado y omisión de datos de partida)
df = df[(df['CNN-3200-CR_0001_MO.RUN'] == 1) & (df['OMITIR'] != 1) & (df['CNN-3200-WIC32149.PV'] > 700)].drop(columns=['RUN_SHIFT', 'CAMBIO_RUN', 'OMITIR'])

df = df.reset_index(drop=True)


In [ ]:
cols = list(df.columns.difference(['Timestamp', 'archivo_origen']))

In [ ]:
df_2 = df

In [ ]:
# Filtrar columnas utiles para el modelo
columnas = [
    'Timestamp',
    'CNN-3200-CR_0001_MO.PWR',
    'CNN-3200-CR_0001_MO.CUR',
    'CNN-3200-FIT32053.PV',
    'CNN-3200-FIT32054.PV',
    'CNN-3200-PIT32031.PV',
    'CNN-3200-PIT32043.PV',
    'CNN-3200-PIT32056.PV',
    'CNN-3200-TIT32045.PV',
    'CNN-3200-TIT32046.PV'
]

df = df[columnas]

# Rellenar valores faltantes
df = df.interpolate(method='linear', limit_direction='both')

In [ ]:
# Guardar el dataset preprocesado
df.to_csv('datos_procesados_vf.csv', index=False)

# Guardar dataset para testeo final
df_test_final.to_csv('data_abril_test_final.csv', index=False)

In [ ]:
columnas_sensores = [
    'CNN-3200-CR_0001_MO.PWR',
    'CNN-3200-CR_0001_MO.CUR',
    'CNN-3200-FIT32053.PV',
    'CNN-3200-FIT32054.PV',
    'CNN-3200-PIT32031.PV',
    'CNN-3200-PIT32043.PV',
    'CNN-3200-PIT32056.PV',
    'CNN-3200-TIT32045.PV',
    'CNN-3200-TIT32046.PV'
]

In [ ]:
palette = sns.color_palette("deep") # Paleta de colores
for sensor in cols:
    df_2[sensor] = df_2[sensor].round(2)
    min_val = df_2[sensor].min()
    max_val = df_2[sensor].max()
    plt.figure(figsize=(20, 4))
    sns.histplot(df_2[sensor], bins=100, stat='density', color=[0.8]*3)
    sns.kdeplot(df_2[sensor], color=palette[3])
    etiquetas = np.round(np.linspace(min_val, max_val, 27), 2)
    plt.xticks(etiquetas)
    plt.title(f'Distribución de {sensor} durante operación')
    plt.xlabel(sensor)
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
palette = sns.color_palette("deep") # Paleta de colores
for sensor in columnas_sensores:
    min_val = df[sensor].min()
    max_val = df[sensor].max()
    plt.figure(figsize=(20, 4))
    sns.histplot(df[sensor], bins=100, stat='density', color=[0.8]*3)
    sns.kdeplot(df[sensor], color=palette[3])
    plt.xticks(np.linspace(min_val, max_val, 27))
    plt.title(f'Distribución de {sensor} durante operación')
    plt.xlabel(sensor)
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
# Calcular la desviación estándar en ventanas de 2 minuto (8 registros)
df_2_std = df_2.copy()
for sensor in cols:
    df_2_std[f'{sensor}_std'] = df_2[sensor].rolling(window=8, min_periods=1).std()

In [ ]:
# Visualizar histogramas de las desviaciones estándar
palette = sns.color_palette("deep")
for sensor in cols:
    min_val = df_2_std[f'{sensor}_std'].min()
    max_val = df_2_std[f'{sensor}_std'].max()
    plt.figure(figsize=(20, 4))
    sns.histplot(df_2_std[f'{sensor}_std'], bins=100, stat='density', color=[0.8]*3)
    sns.kdeplot(df_2_std[f'{sensor}_std'], color=palette[3])
    plt.xticks(np.linspace(min_val, max_val, 27))
    plt.title(f'Distribución de la desviación estándar de {sensor} en ventanas de 1 minuto')
    plt.xlabel(f'{sensor} (Desviación estándar)')
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
# Calcular la desviación estándar en ventanas de 2 minuto (8 registros)
df_std = df.copy()
for sensor in columnas_sensores:
    df_std[f'{sensor}_std'] = df[sensor].rolling(window=8, min_periods=1).std()

In [ ]:
# Visualizar histogramas de las desviaciones estándar
palette = sns.color_palette("deep")
for sensor in columnas_sensores:
    min_val = df_std[f'{sensor}_std'].min()
    max_val = df_std[f'{sensor}_std'].max()
    plt.figure(figsize=(20, 4))
    sns.histplot(df_std[f'{sensor}_std'], bins=100, stat='density', color=[0.8]*3)
    sns.kdeplot(df_std[f'{sensor}_std'], color=palette[3])
    plt.xticks(np.linspace(min_val, max_val, 27))
    plt.title(f'Distribución de la desviación estándar de {sensor} en ventanas de 1 minuto')
    plt.xlabel(f'{sensor} (Desviación estándar)')
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
df_std = df_std.dropna().reset_index(drop=True)

In [ ]:
limites_sensores = {
    'CNN-3200-CR_0001_MO.PWR': {"min": 0, "max": 590},
    'CNN-3200-CR_0001_MO.CUR': {"min": 42, "max": 100},
    'CNN-3200-FIT32053.PV': {"min": 60, "max": 92},
    'CNN-3200-FIT32054.PV': {"min": 106, "max": 115},
    'CNN-3200-PIT32031.PV': {"min": 120, "max": 265.5},
    'CNN-3200-PIT32043.PV': {"min": 77, "max": 131},
    'CNN-3200-PIT32056.PV': {"min": 55, "max": 148},
    'CNN-3200-TIT32045.PV': {"min": 39.7, "max": 47.7},
    'CNN-3200-TIT32046.PV': {"min": 39.8, "max": 46.5}
}

limites_std = {
    'CNN-3200-CR_0001_MO.PWR_std': {"max": 224},
    'CNN-3200-CR_0001_MO.CUR_std': {"max": 26},
    'CNN-3200-FIT32053.PV_std': {"max": 1.5},
    'CNN-3200-FIT32054.PV_std': {"max": 0.9},
    'CNN-3200-PIT32031.PV_std': {"max": 61.5},
    'CNN-3200-PIT32043.PV_std': {"max": 2.7},
    'CNN-3200-PIT32056.PV_std': {"max": 18.5},
    'CNN-3200-TIT32045.PV_std': {"max": 0.85},
    'CNN-3200-TIT32046.PV_std': {"max": 0.81}
}


In [ ]:
# Función para determinar si un registro es posible falla
def detectar_falla(row):
    for sensor, limites in limites_sensores.items():
        if row[sensor] < limites["min"] or row[sensor] > limites["max"]:
            return 1  # Posible falla
    
    for sensor_std, limites in limites_std.items():
        if row[sensor_std] > limites["max"]:
            return 1  # Posible falla
    
    return 0  # Normal

# Aplicar función a cada fila
df_std["Posible_Falla"] = df_std.apply(detectar_falla, axis=1)

# Guardar dataset etiquetado
df_std.to_csv("datos_etiquetados_vf.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

# Seleccionar registros normales (sin fallas)
df_normal = df_std[df_std["Posible_Falla"] == 0].copy()

# Número de fallas a generar (60% del dataset normal)
num_fallas = int(len(df_normal) * 0.60)

# Dividir en tres tipos de fallas sintéticas
num_fallas_1 = int(num_fallas * 0.34)  # Alteraciones en sensor y std
num_fallas_2 = int(num_fallas * 0.33)  # Alteraciones solo en el sensor
num_fallas_3 = num_fallas - num_fallas_1 - num_fallas_2  # Alteraciones solo en la std

# Seleccionar registros aleatorios para cada tipo
fallas_1 = df_normal.sample(n=num_fallas_1, random_state=42).copy()
fallas_2 = df_normal.sample(n=num_fallas_2, random_state=43).copy()
fallas_3 = df_normal.sample(n=num_fallas_3, random_state=44).copy()

# Aplicar modificaciones en los sensores
for sensor, limites in limites_sensores.items():
    desviacion = (limites["max"] - limites["min"]) * 0.4  # Anomalía del 40% del rango normal

    # Tipo 1: Alteraciones en el sensor y std
    fallas_1[sensor] = np.where(
        np.random.rand(len(fallas_1)) > 0.5,
        fallas_1[sensor] + desviacion,  # Aumentar el valor
        fallas_1[sensor] - desviacion   # Disminuir el valor
    )

    # Tipo 2: Alteraciones solo en el sensor (manteniendo std normal)
    fallas_2[sensor] = np.where(
        np.random.rand(len(fallas_2)) > 0.5,
        fallas_2[sensor] + desviacion,  
        fallas_2[sensor] - desviacion
    )

# Aplicar modificaciones en las desviaciones estándar
for sensor_std, limites in limites_std.items():
    aumento_std = limites["max"] * 1.7  # Aumentar 70% sobre el límite máximo

    # Tipo 1: Alteraciones en el sensor y std
    fallas_1[sensor_std] += aumento_std  

    # Tipo 3: Alteraciones solo en std (manteniendo valores normales)
    fallas_3[sensor_std] += aumento_std  

# Etiquetar los registros como fallas
fallas_1["Posible_Falla"] = 1
fallas_2["Posible_Falla"] = 1
fallas_3["Posible_Falla"] = 1

# Combinar dataset original con fallas sintéticas
df_final = pd.concat([df_std, fallas_1, fallas_2, fallas_3]).sort_values(by="Timestamp").reset_index(drop=True)

# Guardar dataset con fallas sintéticas
df_final.to_csv("datos_con_fallas_sinteticas_vf.csv", index=False)

print(f"✅ Se generaron {num_fallas} fallas sintéticas y se guardaron en 'datos_con_fallas_sinteticas_vf.csv'.")


In [ ]:
# Contar registros normales y con fallas
print(df_final["Posible_Falla"].value_counts())

In [ ]:
for sensor in columnas_sensores:
    plt.figure(figsize=(20, 4))

    # Histograma del sensor en operación normal
    sns.histplot(df_final[df_final["Posible_Falla"] == 0][sensor], bins=80, color="blue", label="Normal", stat="density", alpha=0.5)

    # Histograma del sensor en registros de falla
    sns.histplot(df_final[df_final["Posible_Falla"] == 1][sensor], bins=80, color="red", label="Falla", stat="density", alpha=0.5)

    plt.title(f'Comparación de distribución - {sensor}')
    plt.xlabel(sensor)
    plt.ylabel('Frecuencia')
    plt.legend()
    plt.show()


In [ ]:
# Ver cuántos registros de fallas están fuera de los límites
for sensor, limites in limites_sensores.items():
    fuera_limite = df_final[(df_final["Posible_Falla"] == 1) & 
                      ((df_final[sensor] < limites["min"]) | (df_final[sensor] > limites["max"]))]
    print(f"{sensor}: {len(fuera_limite)} registros de fallas fuera de los límites esperados")

In [ ]:
for sensor_std in limites_std.keys():
    print(f"{sensor_std}:")
    print("  Media en operación normal:", df_final[df_final["Posible_Falla"] == 0][sensor_std].mean())
    print("  Media en fallas:", df_final[df_final["Posible_Falla"] == 1][sensor_std].mean(), "\n")


In [ ]:
# Cargar datos con fallas sintéticas
df = pd.read_csv("datos_con_fallas_sinteticas_2.csv", parse_dates=['Timestamp'])

# Ordenar por Timestamp por seguridad
df = df.sort_values(by="Timestamp").reset_index(drop=True)

# Definir columnas de entrada (excluyendo Timestamp y Posible_Falla)
columnas_sensores = [col for col in df.columns if col not in ['Timestamp', 'Posible_Falla']]

# Normalización (MinMaxScaler entre 0 y 1)
scaler = MinMaxScaler()
df[columnas_sensores] = scaler.fit_transform(df[columnas_sensores])
joblib.dump(scaler, 'scaler.pkl')
# Definir ventana de tiempo para LSTM (ej: 10 registros = 2.5 minutos de contexto)
ventana = 10

# Crear secuencias de datos para LSTM
X, y = [], []
for i in range(len(df) - ventana):
    X.append(df[columnas_sensores].iloc[i:i+ventana].values)  # 10 registros anteriores
    y.append(df["Posible_Falla"].iloc[i+ventana])  # Predicción para el siguiente registro

# Convertir a arrays de NumPy
X = np.array(X)
y = np.array(y)

# Dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"✅ Datos listos para el modelo LSTM:")
print(f"  - X_train shape: {X_train.shape}")  # (num_samples, ventana, num_features)
print(f"  - X_test shape: {X_test.shape}")
print(f"  - y_train shape: {y_train.shape}")
print(f"  - y_test shape: {y_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Definir modelo LSTM
modelo = Sequential([
    LSTM(64, return_sequences=True, input_shape=(10, 18)),  # Capa LSTM con 64 neuronas
    Dropout(0.2),  # Regularización
    LSTM(32, return_sequences=False),  # Segunda capa LSTM
    Dropout(0.2),
    Dense(16, activation='relu'),  # Capa densa
    Dense(1, activation='sigmoid')  # Salida con probabilidad de falla
])

# Compilar el modelo
modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen de la arquitectura
modelo.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir un callback para detener el entrenamiento si la validación no mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Entrenar el modelo
history = modelo.fit(
    X_train, y_train,
    epochs=50,  # Número de épocas (ajustable)
    batch_size=64,  # Tamaño de lote (ajustable)
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Guardar el modelo entrenado
modelo.save("modelo_lstm_fallas.h5")

print("✅ Entrenamiento completado y modelo guardado como 'modelo_lstm_fallas.h5'")


In [ ]:
import joblib

joblib.dump(modelo, "modelo_lstm_fallas.pkl")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Predecir en el conjunto de prueba
y_pred = (modelo.predict(X_test) > 0.5).astype("int32")

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()

# Reporte de clasificación
print(classification_report(y_test, y_pred, target_names=["Normal", "Falla"]))


In [ ]:
# Ajustar el umbral a 0.3 en vez de 0.5
umbral = 0.3
y_pred_ajustado = (modelo.predict(X_test) > umbral).astype("int32")

# Nueva matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_ajustado)

# Mostrar la nueva matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title(f"Matriz de Confusión con umbral {umbral}")
plt.show()

# Nuevo reporte de clasificación
print(classification_report(y_test, y_pred_ajustado, target_names=["Normal", "Falla"]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# Definir el modelo CNN + LSTM
modelo_cnn_lstm = Sequential([
    # Capa CNN para extraer características espaciales
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(10, 18)),
    MaxPooling1D(pool_size=2),  # Reduce la dimensionalidad

    # Capa LSTM para capturar patrones temporales
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),

    # Capas densas
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Salida con probabilidad de falla
])

# Compilar el modelo
modelo_cnn_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
modelo_cnn_lstm.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir Early Stopping para evitar sobreentrenamiento
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo CNN + LSTM
history = modelo_cnn_lstm.fit(
    X_train, y_train,
    epochs=50,  # Número de épocas (ajustable)
    batch_size=256,  # Tamaño de lote
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Guardar el modelo entrenado
modelo_cnn_lstm.save("modelo_cnn_lstm_fallas.h5")

print("✅ Entrenamiento completado y modelo guardado como 'modelo_cnn_lstm_fallas.h5'")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Predecir en el conjunto de prueba con el umbral original (0.5)
y_pred_cnn_lstm = (modelo_cnn_lstm.predict(X_test) > 0.5).astype("int32")

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_cnn_lstm)

# Mostrar la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - Modelo CNN + LSTM")
plt.show()

# Reporte de clasificación
print(classification_report(y_test, y_pred_cnn_lstm, target_names=["Normal", "Falla"]))


In [ ]:
# Ajustar umbral de clasificación a 0.3
umbral = 0.3
y_pred_cnn_lstm_ajustado = (modelo_cnn_lstm.predict(X_test) > umbral).astype("int32")

# Matriz de confusión con nuevo umbral
conf_matrix = confusion_matrix(y_test, y_pred_cnn_lstm_ajustado)

# Mostrar la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title(f"Matriz de Confusión - CNN + LSTM (Umbral {umbral})")
plt.show()

# Reporte de clasificación
print(classification_report(y_test, y_pred_cnn_lstm_ajustado, target_names=["Normal", "Falla"]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization

# Definir el modelo optimizado CNN + BiLSTM
modelo_opt = Sequential([
    # Primera capa CNN
    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(10, 18)),
    BatchNormalization(),  # Normalización para estabilidad
    MaxPooling1D(pool_size=2),

    # Segunda capa CNN para mayor profundidad
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    # Capa LSTM Bidireccional
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),

    # Segunda capa LSTM Bidireccional
    Bidirectional(LSTM(32, return_sequences=False)),
    Dropout(0.3),

    # Capas densas
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Salida con probabilidad de falla
])

# Compilar el modelo
modelo_opt.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                   loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
modelo_opt.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir Early Stopping para evitar sobreentrenamiento
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo optimizado
history_opt = modelo_opt.fit(
    X_train, y_train,
    epochs=50,  # Número de épocas (ajustable)
    batch_size=256,  # Tamaño de lote
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Guardar el modelo entrenado
modelo_opt.save("modelo_optimizado_cnn_bilstm.h5")

print("✅ Entrenamiento completado y modelo guardado como 'modelo_optimizado_cnn_bilstm.h5'")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Predecir en el conjunto de prueba con el umbral original (0.5)
y_pred_opt = (modelo_opt.predict(X_test) > 0.5).astype("int32")

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_opt)

# Mostrar la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - Modelo Optimizado CNN + BiLSTM")
plt.show()

# Reporte de clasificación
print(classification_report(y_test, y_pred_opt, target_names=["Normal", "Falla"]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization

# Definir el modelo optimizado con más capas LSTM
modelo_optimizado = Sequential([
    # Primera capa CNN con más filtros
    Conv1D(filters=256, kernel_size=3, activation='relu', padding='same', input_shape=(10, 18)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    # Segunda capa CNN con menos filtros
    Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    # Primera capa BiLSTM
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),

    # Segunda capa BiLSTM
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),

    # Tercera capa LSTM tradicional
    LSTM(32, return_sequences=False),
    Dropout(0.3),

    # Capas densas
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Salida con probabilidad de falla
])

# Compilar el modelo
modelo_optimizado.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
                          loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
modelo_optimizado.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir Early Stopping para evitar sobreentrenamiento
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo mejorado
history_mejorado = modelo_optimizado.fit(
    X_train, y_train,
    epochs=50,  # Número de épocas
    batch_size=256,  # Tamaño de lote
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Guardar el modelo entrenado
modelo_optimizado.save("modelo_mejorado_cnn_bilstm.h5")

print("✅ Entrenamiento completado y modelo guardado como 'modelo_mejorado_cnn_bilstm.h5'")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Predecir en el conjunto de prueba con el umbral original (0.5)
y_pred_mejorado = (modelo_optimizado.predict(X_test) > 0.5).astype("int32")

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_mejorado)

# Mostrar la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - Modelo Mejorado CNN + BiLSTM")
plt.show()

# Reporte de clasificación
print(classification_report(y_test, y_pred_mejorado, target_names=["Normal", "Falla"]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam

# Definir Focal Loss con menor gamma
import tensorflow.keras.backend as K
def focal_loss(gamma=1.5, alpha=0.25):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * y_true * K.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return K.mean(loss)
    return loss

# Definir el modelo LSTM corregido
modelo_lstm_corr = Sequential([
    LSTM(128, return_sequences=True, input_shape=(10, 18)),
    LayerNormalization(),
    Dropout(0.2),

    LSTM(64, return_sequences=True),
    LayerNormalization(),
    Dropout(0.2),

    LSTM(32, return_sequences=False),
    LayerNormalization(),
    Dropout(0.2),

    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Salida con probabilidad de falla
])

# Compilar el modelo con la nueva Focal Loss
modelo_lstm_corr.compile(optimizer=Adam(learning_rate=0.0005),
                         loss=focal_loss(gamma=1.5, alpha=0.25),
                         metrics=['accuracy'])

# Mostrar el resumen del modelo corregido
modelo_lstm_corr.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir Early Stopping para evitar sobreentrenamiento
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo corregido
history_lstm_corr = modelo_lstm_corr.fit(
    X_train, y_train,
    epochs=50,  # Número de épocas
    batch_size=256,  # Tamaño de lote
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Guardar el modelo entrenado
modelo_lstm_corr.save("modelo_lstm_corregido.h5")

print("✅ Entrenamiento completado y modelo guardado como 'modelo_lstm_corregido.h5'")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo basado en el artículo
modelo_lstm_articulo = Sequential([
    LSTM(100, activation='relu', return_sequences=True, input_shape=(10, 18)),
    Dropout(0.2),

    LSTM(50, activation='relu', return_sequences=True),
    Dropout(0.2),

    LSTM(25, activation='relu', return_sequences=False),
    Dropout(0.2),

    Dense(50, activation='relu'),
    Dropout(0.2),

    Dense(1, activation='sigmoid')  # Salida con probabilidad de falla
])

# Compilar el modelo con función de pérdida 'binary_crossentropy'
modelo_lstm_articulo.compile(optimizer=Adam(learning_rate=0.0005),
                             loss='binary_crossentropy',
                             metrics=['accuracy'])

# Mostrar el resumen del modelo
modelo_lstm_articulo.summary()


In [ ]:
# Definir Early Stopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history_lstm_articulo = modelo_lstm_articulo.fit(
    X_train, y_train,
    epochs=50,  # Número de épocas
    batch_size=64,  # Tamaño de lote reducido
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Guardar el modelo entrenado
modelo_lstm_articulo.save("modelo_lstm_articulo.h5")

print("✅ Entrenamiento completado y modelo guardado como 'modelo_lstm_articulo.h5'")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Predicciones
y_pred_prob = modelo_lstm_articulo.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Umbral de 0.5 para clasificar

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Mostrar matriz de confusión
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - Modelo LSTM Adaptado")
plt.show()

# Reporte de métricas
print(classification_report(y_test, y_pred))


In [ ]:
# Ajustar el umbral de clasificación
umbral = 0.3
y_pred_ajustado = (y_pred_prob > umbral).astype(int)

# Nueva matriz de confusión
cm_ajustado = confusion_matrix(y_test, y_pred_ajustado)

# Graficar la nueva matriz de confusión
plt.figure(figsize=(6, 4))
sns.heatmap(cm_ajustado, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title(f"Matriz de Confusión - Umbral {umbral}")
plt.show()

# Reporte de métricas
print(classification_report(y_test, y_pred_ajustado))


In [ ]:
# Crear el DataFrame con datos de diciembre
df_nuevos = df_2024.copy()
fecha_inicio = pd.to_datetime('2024-12-01')
df_nuevos = df_nuevos[df_nuevos['Timestamp'] < fecha_inicio].reset_index(drop=True)

# Filtrar solo cuando el chancador está en RUN = 1
df_nuevos = df_nuevos[df_nuevos["CNN-3200-CR_0001_MO.RUN"] == 1]

# Omitir los primeros 8 registros tras cambio de estado RUN 0 → 1
df_nuevos["RUN_shift"] = df_nuevos["CNN-3200-CR_0001_MO.RUN"].shift(1).fillna(0)
df_nuevos["RUN_cambio"] = (df_nuevos["CNN-3200-CR_0001_MO.RUN"] == 1) & (df_nuevos["RUN_shift"] == 0)
df_nuevos["omit"] = df_nuevos["RUN_cambio"].rolling(window=8, min_periods=1).max()
df_nuevos = df_nuevos[(df_nuevos["CNN-3200-CR_0001_MO.RUN"] == 1) & 
                       (df_nuevos['omit'] != 1) & 
                       (df_nuevos['CNN-3200-WIC32149.PV'] > 150)]

# Eliminar columnas auxiliares
df_nuevos.drop(columns=["RUN_shift", "RUN_cambio", "omit"], inplace=True)

# Generar valores sintéticos para el sensor faltante
rango = [110, 110.3, 111, 110.5, 112, 110.2, 113, 110.8, 112, 114, 112.6, 115, 114.2, 111, 110, 110.11, 114]
df_nuevos['CNN-3200-FIT32054.PV'] = rango * (len(df_nuevos) // len(rango)) + rango[:len(df_nuevos) % len(rango)]

# Rellenar valores faltantes
df_nuevos = df_nuevos.interpolate(method='linear', limit_direction='both')

# Seleccionar sensores y calcular desviaciones estándar
columnas_sensores = [
    'CNN-3200-CR_0001_MO.PWR',
    'CNN-3200-CR_0001_MO.CUR',
    'CNN-3200-FIT32053.PV',
    'CNN-3200-FIT32054.PV',
    'CNN-3200-PIT32031.PV',
    'CNN-3200-PIT32043.PV',
    'CNN-3200-PIT32056.PV',
    'CNN-3200-TIT32045.PV',
    'CNN-3200-TIT32046.PV'
]

for col in columnas_sensores:
    df_nuevos[f"{col}_std"] = df_nuevos[col].rolling(window=4, min_periods=1).std()

# Asegurar que las columnas sean las mismas que en el entrenamiento
columnas_sensores_std = columnas_sensores + [f"{col}_std" for col in columnas_sensores]
df_nuevos = df_nuevos[columnas_sensores_std]

# 🔹 Normalizar con el scaler guardado en el entrenamiento
scaler = joblib.load("scaler.pkl")  # Cargar el scaler usado en entrenamiento
df_nuevos_scaled = scaler.transform(df_nuevos)  # Solo transformar, NO fit

# 🔹 Crear ventanas de tiempo (10 registros por cada predicción)
secuencia = 10  # Mismo tamaño usado en el modelo
X_nuevos = np.array([df_nuevos_scaled[i - secuencia : i] for i in range(secuencia, len(df_nuevos))])

# Verificar dimensiones
print(f"Shape de X_nuevos: {X_nuevos.shape}")
print(f"Shape de X_train: {X_train.shape}")

# 🔹 Cargar el modelo
modelo = load_model("modelo_lstm_articulo.h5")

# 🔹 Hacer predicciones
y_nuevos_prob = modelo.predict(X_nuevos)

# 🔹 Aplicar umbral ajustado
umbral = 0.3
y_nuevos_pred = (y_nuevos_prob > umbral).astype(int)

print("✅ Predicciones completadas.")


In [ ]:
print("Distribución de predicciones:")
unique, counts = np.unique(y_nuevos_pred, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
sns.histplot(y_nuevos_prob, bins=50, kde=True)
plt.xlabel("Probabilidad de Falla")
plt.ylabel("Frecuencia")
plt.title("Distribución de Probabilidades de Falla")
plt.show()

In [ ]:
nuevo_umbral = 0.7
y_nuevos_pred_ajustado = (y_nuevos_prob > nuevo_umbral).astype(int)

unique, counts = np.unique(y_nuevos_pred_ajustado, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_nuevos_pred, y_nuevos_pred_ajustado)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Falla"], yticklabels=["Normal", "Falla"])
plt.xlabel("Predicción Ajustada")
plt.ylabel("Predicción Original")
plt.title(f"Matriz de Confusión - Umbral Ajustado a {nuevo_umbral}")
plt.show()
